<a href="https://colab.research.google.com/github/sumitasharma/MachineLearning/blob/master/vgg16_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.applications.vgg16 import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [0]:
vgg_model = VGG16()

In [0]:
vgg_model.summary()

In [0]:
type(vgg_model)

In [0]:
new_model = Sequential()
for layer in vgg_model.layers[:-1]:
  new_model.add(layer)

In [0]:
type(new_model)

In [0]:
new_model.summary()

In [0]:
for layer in new_model.layers:
  layer.trainable = False
  

In [0]:
new_model.add(Dense(2, activation = 'softmax'))

In [0]:
new_model.summary()

In [0]:
new_model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
 !curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
 !unzip kagglecatsanddogs_3367a.zip

In [0]:
from tensorflow.keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='/contents/log/cats_dogs_log')

In [0]:
!mkdir -p /content/PetImages/train/cat
!mkdir -p /content/PetImages/test/cat
!mkdir -p /content/PetImages/valid/cat
!mkdir -p /content/PetImages/train/dog
!mkdir -p /content/PetImages/test/dog
!mkdir -p /content/PetImages/valid/dog

In [0]:
%cd /content/PetImages/Cat

In [0]:
#!ls /content/PetImages/Cat | shuf -n 20 | xargs -i mv {} /content/PetImages/test/cat


!ls  | shuf -n 20  | xargs -i ls {} /content/PetImages/test/cat

In [0]:
!ls  | shuf -n 20| xargs -i mv {} /content/PetImages/valid/cat

In [0]:
!ls  | shuf -n 20| xargs -i mv {} /content/PetImages/test/cat

In [0]:
!ls  | xargs -i mv {} /content/PetImages/train/cat

In [0]:
%cd /content/PetImages/Dog

In [0]:
!ls  | shuf -n 40| xargs -i mv {} /content/PetImages/valid/dog

In [0]:
!ls  | shuf -n 20| xargs -i mv {} /content/PetImages/test/dog

In [0]:
!ls  | xargs -i mv {} /content/PetImages/train/dog

In [0]:
train_generator = ImageDataGenerator().flow_from_directory(
        '/content/PetImages/train',
        target_size=(224, 224),
        batch_size=32,
        classes=['cat','dog'])
test_generator = ImageDataGenerator().flow_from_directory(
        '/content/PetImages/test',
        target_size=(224, 224),
        batch_size=10,
        classes=['cat','dog'])
valid_generator = ImageDataGenerator().flow_from_directory(
        '/content/PetImages/valid',
        target_size=(224, 224),
        batch_size=10,
        classes=['cat','dog'])

In [0]:
images, labels = next(trian_generator)

In [0]:
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [0]:
plots(images, titles=labels)

In [0]:
new_model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=5,
        validation_data=valid_generator,
        validation_steps=50)

In [0]:
images, labels = next(test_generator)
plots(images, titles=labels)

predict = new_model.predict_generator(test_generator,steps=1)
predict = list(map(int, np.round(predict[:,0])))
labels = list(map(int,labels[:,0]))

In [0]:
print(labels)
print(predict)

In [0]:
confusion_matrix(labels, predict)

In [0]:
!mkdir -p /content/PetImages/test3/cat
!mkdir -p /content/PetImages/test3/dog


In [0]:
%cd /content/PetImages/train/cat
!ls  | shuf -n 1  | xargs -i mv  {} /content/PetImages/test3/cat
%cd /content/PetImages/train/dog
!ls  | shuf -n 1  | xargs -i mv  {} /content/PetImages/test3/dog

In [0]:
test3_generator = ImageDataGenerator().flow_from_directory(
        '/content/PetImages/test3',
        target_size=(224, 224),
        batch_size=1,
        classes=['cat','dog'])
predict = new_model.predict_generator(test3_generator,steps=1)
print(predict)

In [0]:
x,y = next(test3_generator)
plots(x, titles=y)

In [0]:
predict = new_model.predict_generator(test2_generator,steps=1)
predict = list(map(int, np.round(predict[:,0])))
print(predict)


In [0]:
from keras.models import load_model

new_model.save('/content/vgg16_fine_tuning_data_generator_cat_dog.h5')